In [33]:
import fastavro
from sqlalchemy import create_engine, MetaData, Table, select, Column,Integer, String
import datetime

In [34]:
date_backup=datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
engine = create_engine('mysql+pymysql://root@localhost/flaskmysql')
metadata = MetaData()
table = Table('jobs', metadata,
    Column('id', Integer, primary_key=True),
    Column('job', String(100)),
)


In [35]:
metadata.create_all(engine)
query = table.insert()

In [36]:
print(query)

INSERT INTO jobs (id, job) VALUES (:id, :job)


In [37]:
with open('jobs.avro', 'rb') as f:
    schema = {
        'type': 'record',
        'name': 'Job',
        'fields': [
            {'name': 'id', 'type': 'int'},
            {'name': 'job', 'type': 'string'},
        ]
    }
    with engine.begin() as conn:
        for record in fastavro.reader(f, schema):
            conn.execute(query, record)